In [ ]:
from ultralytics import YOLO
import cv2

In [ ]:
video_path = "ma_video.mp4"  # Chemin vers ta vidéo
output_path = "output_yolo_risk.mp4"

# Pondération par type d'objet
risk_weights = {
    "person": 1.0,
    "bicycle": 0.7,
    "car": 0.5,
    "motorcycle": 0.5,
    "truck": 0.5,
    "bus": 0.5
}

In [ ]:
model = YOLO("yolov11n.pt") 

In [ ]:
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Impossible d'ouvrir la vidéo : {video_path}")
    exit()

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

out = cv2.VideoWriter(output_path,
                      cv2.VideoWriter_fourcc(*'mp4v'),
                      fps,
                      (frame_width, frame_height))

In [ ]:
def compute_risk(box, label, frame_width, frame_height):
    x1, y1, x2, y2 = box
    area = (x2 - x1) * (y2 - y1) / (frame_width * frame_height) 
    center_x = (x1 + x2) / 2
    position_factor = 1 - abs(center_x - frame_width/2) / (frame_width/2)
    weight = risk_weights.get(label, 0.2)
    return area * weight * position_factor

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, stream=True)

    frame_risk = 0 

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            label = model.names[cls]
            frame_risk += compute_risk((x1, y1, x2, y2), label, frame_width, frame_height)
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} {conf:.2f}",
                        (int(x1), int(y1) - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    frame_risk_normalized = min(frame_risk, 1.0)  
    cv2.putText(frame, f"Score risque externe: {frame_risk_normalized:.2f}",
                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)

    out.write(frame)
    cv2.imshow("YOLOv11 + Risque Externe", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
